In [37]:
import pandas as pd

data_dir = "./Other_Datasets/ANERcorp/"
name = "train"

bad_words = {'ARTICLE', 'REVISIT', 'ARTICLE', 'article', 'REVIIST', 'ARTICE', 'ARTICLE', 'FIX', 'ID', ' ', 'REVISSIT', 'REVISIIT'}

sentences = []
sentences_labels = []

with open(data_dir + f"{name}.txt", 'r') as file:
    sentence = []  # Initialize as a list
    each_word_label = []
    for i, line in enumerate(file):
        # Check if the line is not empty
        if line.strip():
            parts = line.split(' ')
            # Check if parts has enough elements
            if len(parts) > 1:
                word, label = parts[0], parts[1].strip()
                # Check if the word is not in the set of bad words
                if label not in bad_words:
                    if(word != '.'):
                        sentence.append(word)
                        each_word_label.append(label)

                # If the word is a period, it indicates the end of a sentence
                if '.' == word.strip():
                    if sentence:
                        sentences.append(' '.join(sentence))
                        sentences_labels.append(each_word_label)
                        sentence = []
                        each_word_label = []

# Now you have sentences and their corresponding labels
df = pd.DataFrame({'sentence': sentences, 'labels': sentences_labels})


In [38]:
df.loc[1]['sentence'] + '. كلمة'

'وقال رئيس الاتحاد برند جوتشولك عند إعلان آخر تقرير سنوي للاتحاد إن مستقبل السوق مازال يفتقر إلي الخطوط الواضحة. كلمة'

In [39]:
new_df = pd.DataFrame()

for i in range(len(df)):
    words = df['sentence'][i].split(' ')
    labels = df['labels'][i]
    num_words = len(words)
    
    for j in range(num_words):
        new_row = pd.DataFrame({'label': [labels[j]], 'word': [words[j]], 'sentence': df['sentence'][i]})
        new_df = pd.concat([new_df, new_row], ignore_index=True)


In [40]:
new_df['label'].unique()

array(['B-LOC', 'B-ORG', 'I-ORG', 'O', 'B-PERS', 'I-PERS', 'I-LOC',
       'B-MISC', 'I-MISC'], dtype=object)

In [31]:
new_df.to_csv(data_dir + f"{name}.csv", index=False, header=False)

In [41]:
x = 67

y = '[MASK]' + ' ' + new_df.loc[x]['sentence'] + '. كلمة ' + new_df.loc[x]['word'] + ' هي؟ '
y

'[MASK] وعلي الرغم من أنه قال أنه يتوقع أن تظل صادرات السيارات عند مستوي مرتفع هذا العام فإنه يبدو من غير المحتمل أن تصل إلي مستوي سجل نموها عام 2001 عندما زادت صادرات سيارات الركاب بنسبة ستة في المائة لتصل إلي 6.3 مليون سيارة. كلمة صادرات هي؟ '

In [42]:
x= 150

'('+ new_df.loc[x]['sentence'] + ')' + ' '  +  ' كلمة ' + new_df.loc[x]['word'] + ' هي؟ ' +'[MASK]'

'(وقال أنه يتعين أن يصل عدد السيارات الجديدة في ألمانيا هذا العام إلي حوالي 2.3 مليون سيارة بالمقارنة بـ34.3 مليون سيارة في العام الماضي)  كلمة بـ34.3 هي؟ [MASK]'

In [43]:
edited_df = new_df.copy()
label_mapping = {
    'B-LOC': 'LOC',
    'I-LOC': 'LOC',
    'B-PERS': 'PERS',
    'I-PERS': 'PERS',
    'B-ORG': 'ORG',
    'I-ORG': 'ORG',
    'B-MISC': 'MISC',
    'I-MISC': 'MISC',
    'O': 'O'
}

# Apply the mapping
edited_df['label'] = new_df['label'].map(label_mapping)



In [44]:
edited_df['label'].unique()


array(['LOC', 'ORG', 'O', 'PERS', 'MISC'], dtype=object)

In [45]:
data_dir = "./Other_Datasets/ANERcorpEdited/"

edited_df.to_csv(data_dir + f"{name}.csv", index=False, header=False)